# Preprocessing

This notebook covers the preprocessing for the cleaned data. The preprocessing will include PCA, and feature selection. It will have various pipelines that can be used to train different sets of processed data to compare the results of a machine learning model. 

Import Libraries

In [100]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb

import warnings

warnings.filterwarnings('ignore')

Before we begin preprocessing, we will run some simple models to determine a baseline. This baseline can then be used as a benchmark after completing other preprocessing steps.

### Importing Cleaned Data

In [15]:
kois = pd.read_csv('..\data\kois_cleaned.csv', index_col=0)
kois.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_disposition_encoded
rowid,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,1
2,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,1
3,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,...,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,2
4,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,2
5,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,...,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,1


In [43]:
#exclude error columns with .describe()
kois.loc[:, ~kois.columns.str.contains('_err')].describe()


,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag,koi_disposition_encoded
count,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9.007000e+03,9007.000000,9007.000000,9.007000e+03,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000,9007.000000
mean,0.167203,0.235039,0.197402,0.122238,75.093189,165.792699,0.722475,5.636540,2.293444e+04,102.880978,1077.611302,7.325554e+03,262.226624,5704.450094,4.310499,1.699245,292.047428,43.828852,14.268715,1.265238
std,0.373178,0.424047,0.398060,0.327579,1375.035264,67.589891,3.296483,6.409275,8.078573e+04,3109.455176,843.714627,1.584377e+05,803.003738,797.670670,0.430133,5.622511,4.760429,3.598752,1.348588,0.821739
min,0.000000,0.000000,0.000000,0.000000,0.241843,120.515914,0.000000,0.167000,4.500000e+00,0.140000,25.000000,0.000000e+00,1.600000,2661.000000,0.047000,0.109000,279.852720,36.577381,6.966000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,2.741232,132.773544,0.196000,2.454245,1.603000e+02,1.400000,537.000000,1.960500e+01,12.100000,5301.000000,4.215000,0.827000,288.660770,40.810164,13.469000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,9.449426,137.157520,0.534000,3.823530,4.183000e+02,2.380000,875.000000,1.383400e+02,23.000000,5761.000000,4.440000,0.997000,292.249020,43.720329,14.534000,2.000000
75%,0.000000,0.000000,0.000000,0.000000,37.597864,170.545120,0.886000,6.308000,1.413550e+03,14.375000,1372.000000,8.359150e+02,76.650000,6116.000000,4.544000,1.347000,295.856400,46.722311,15.319000,2.000000
max,1.000000,1.000000,1.000000,1.000000,129995.778400,1472.522306,100.806000,138.540000,1.541400e+06,200346.000000,14667.000000,1.094755e+07,9054.700000,15896.000000,5.364000,180.013000,301.720760,52.336010,19.065000,2.000000


In [49]:
print(kois.shape)
kois.isnull().sum()

(9007, 40)


koi_fpflag_nt              0
koi_fpflag_ss              0
koi_fpflag_co              0
koi_fpflag_ec              0
koi_period                 0
koi_period_err1            0
koi_period_err2            0
koi_time0bk                0
koi_time0bk_err1           0
koi_time0bk_err2           0
koi_impact                 0
koi_impact_err1            0
koi_impact_err2            0
koi_duration               0
koi_duration_err1          0
koi_duration_err2          0
koi_depth                  0
koi_depth_err1             0
koi_depth_err2             0
koi_prad                   0
koi_prad_err1              0
koi_prad_err2              0
koi_teq                    0
koi_insol                  0
koi_insol_err1             0
koi_insol_err2             0
koi_model_snr              0
koi_steff                  0
koi_steff_err1             0
koi_steff_err2             0
koi_slogg                  0
koi_slogg_err1             0
koi_slogg_err2             0
koi_srad                   0
koi_srad_err1 

### Splitting Data

In [17]:
#separate our target variable from the rest of the data
y = kois['koi_disposition_encoded']
X = kois.drop(['koi_disposition_encoded'], axis=1)

In [29]:
#print the percentage of values in each class
print(y.value_counts(normalize=True))
y.describe()

koi_disposition_encoded
2    0.505385
1    0.254469
0    0.240147
Name: proportion, dtype: float64


count    9007.000000
mean        1.265238
std         0.821739
min         0.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         2.000000
Name: koi_disposition_encoded, dtype: float64

In [18]:
#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #use a test size of 20%

### Instantiating Models

In [19]:
#Decision tree classifier
dtc = DecisionTreeClassifier()

#Random forest classifier
rfc = RandomForestClassifier()

#Logistic regression classifier
logreg = LogisticRegression()

#Support vector machine classifier
svc = SVC()

#K-nearest neighbors classifier
knn = KNeighborsClassifier()

#XGBoost classifier
xgbc = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

### Train the Models

In [22]:
#train Decision tree classifier
dtc.fit(X_train, y_train)

#train Random forest classifier
rfc.fit(X_train, y_train)

#train Logistic regression classifier
logreg.fit(X_train, y_train)

#train Support vector machine classifier
svc.fit(X_train, y_train)

#train K-nearest neighbors classifier
knn.fit(X_train, y_train)

#train XGBoost classifier
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

### Check Model Performance

In [23]:
#Decision tree classifier score
dtc_score = dtc.score(X_test, y_test)

#Random forest classifier score
rfc_score = rfc.score(X_test, y_test)

#Logistic regression classifier score
logreg_score = logreg.score(X_test, y_test)

#Support vector machine classifier score
svc_score = svc.score(X_test, y_test)

#K-nearest neighbors classifier score
knn_score = knn.score(X_test, y_test)

#XGBoost classifier score
xgbc_score = xgbc.score(X_test, y_test)

In [30]:
#Print the scores

# Create a dictionary with the model names and their scores
model_scores = {
    "Decision Tree": dtc_score,
    "Random Forest": rfc_score,
    "Logistic Regression": logreg_score,
    "SVM": svc_score,
    "KNN": knn_score,
    "XGBoost": xgbc_score
}

# Convert the dictionary to a DataFrame
scores_df = pd.DataFrame(list(model_scores.items()), columns=['Model', 'Accuracy Score'])

# Sort the DataFrame by 'Accuracy Score' in descending order
scores_df = scores_df.sort_values(by='Accuracy Score', ascending=False)

# Display the DataFrame
print(scores_df)

                 Model  Accuracy Score
5              XGBoost        0.893452
1        Random Forest        0.891787
0        Decision Tree        0.849057
4                  KNN        0.623751
2  Logistic Regression        0.529967
3                  SVM        0.490566


Now that we have the above scores, we have a benchmark to work with. We also know that the median is 2 for the target variable. And if you chose the median for every prediction you would be right approximately 50% of the time. Looking at the scores from our models, we can now see which models had the greatest gains over simply choosing the median every time. 

We can also see which models performed worse than our simplest guess. 

Before we cast judgment on the performance of these models, we have to remember that some are more sensitive to feature scaling than others. For example, Logistic Regression, SVM, and KNN all benefit greatly from feature scaling, while Decision Trees, Random Forest, and XGBoost all are typically invariant to feature scaling. 

What this tells us is that in order to utilize some of these models properly we should perform some sort of feature scaling on the data.

### Feature Scaling

For SVM and Logistic Regression, we will use the Z-score normalization or `StandardScaler()` from the Scikit Learn library. 

For KNN, we will use Min-Max scaling (normalization) or the `MinMaxScaler()` also from Scikit Learn. 

In [54]:
#fit the scalers to the train and test sets separately and transform both
scaler = StandardScaler()

X_train_stndrd = scaler.fit_transform(X_train)
X_test_stndrd = scaler.transform(X_test)

minmax = MinMaxScaler()

X_train_minmax = minmax.fit_transform(X_train)
X_test_minmax = minmax.transform(X_test)

### Retrain Models

We can now use the scaled data to retrain our models. We can compare it with our tree models without scaled data, as well as pass the scaled data through the tree models to evaluate performance. These scores can act as our new benchmarks.

In [35]:
#training our tree and ensemble models on the scaled data
#train Decision tree classifier
dtc.fit(X_train_stndrd, y_train)

#train Random forest classifier
rfc.fit(X_train_stndrd, y_train)

#train XGBoost classifier
xgbc.fit(X_train_stndrd, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [36]:
#Training the remaining models on their preferred scaling method
#train Logistic regression classifier
logreg.fit(X_train_stndrd, y_train)

#train Support vector machine classifier
svc.fit(X_train_stndrd, y_train)

#train K-nearest neighbors classifier
knn.fit(X_train_minmax, y_train)

KNeighborsClassifier()

In [39]:
#Decision tree classifier score
dtc_score = dtc.score(X_test_stndrd, y_test)

#Random forest classifier score
rfc_score = rfc.score(X_test_stndrd, y_test)

#Logistic regression classifier score
logreg_score = logreg.score(X_test_stndrd, y_test)

#Support vector machine classifier score
svc_score = svc.score(X_test_stndrd, y_test)

#K-nearest neighbors classifier score
knn_score = knn.score(X_test_minmax, y_test)

#XGBoost classifier score
xgbc_score = xgbc.score(X_test_stndrd, y_test)

In [40]:
#Print the scores

# Create a dictionary with the model names and their scores
model_scores_scaled = {
    "Decision Tree": dtc_score,
    "Random Forest": rfc_score,
    "Logistic Regression": logreg_score,
    "SVM": svc_score,
    "KNN": knn_score,
    "XGBoost": xgbc_score
}

# Convert the dictionary to a DataFrame
scores_scaled = pd.DataFrame(list(model_scores_scaled.items()), columns=['Model-Scaling', 'Accuracy Score'])

# Sort the DataFrame by 'Accuracy Score' in descending order
scores_scaled = scores_scaled.sort_values(by='Accuracy Score', ascending=False)

# Display the DataFrame for both sets of scores
print(scores_df)
print(scores_scaled)

                 Model  Accuracy Score
5              XGBoost        0.893452
1        Random Forest        0.891787
0        Decision Tree        0.849057
4                  KNN        0.623751
2  Logistic Regression        0.529967
3                  SVM        0.490566
         Model-Scaling  Accuracy Score
5              XGBoost        0.893452
1        Random Forest        0.892342
2  Logistic Regression        0.876249
3                  SVM        0.869589
0        Decision Tree        0.849057
4                  KNN        0.809101


We can see huge improvements just by applying scaling to our data for the the models that are typically sensitive to it. Our ensemble models maintained their performance rankings with only a minor change in the random forest performance and no change in the performance of XGBoost. 

Logistic Regression had the largest gain of 0.35. Every model is now performing in the 80% territory which is fairly strong when compared to our absolute base benchmark of always choosing the median. 

### Outliers

From the data cleaning and EDA notebook, we saw that there were potential outliers, not only in the data, but also in the error values. We opted to leave the outliers in due to a lack of domain knowledge. But as part of our preprocessing we can not remove some of those extreme values and compare performance.

We will start off with very conservative outlier removal, to see if this improves our performance. Since we saw vast improvements from our scaling, we will also scale the data once outliers have been removed. From what we saw in our data exploration notebook, most of the variables were left skewed, with their most extreme values being greater than the mean. Eliminating outliers outside of IQR + 3 * $\sigma$ can help remove these extreme values. 

In [70]:
# Calculate the Z-score for each feature in the DataFrame
z_scores = (kois - kois.mean()) / kois.std()

# Identify rows where any feature has a Z-score greater than 4
outliers = (z_scores > 3)

# Filter those rows from the DataFrame
kois_no_outliers = kois[~outliers.any(axis=1)]

In [71]:
#print the shape of our original data and our new dataset with no outliers
kois_no_outliers.shape

(7418, 40)

We can see from the above that we have eliminated about 1050 values that were categorized as outliers. We can now split our new data set, and apply our feature scaling.

In [72]:
#separate our target variable from the rest of the data
y_no_outliers = kois_no_outliers['koi_disposition_encoded']
X_not_outliers = kois_no_outliers.drop(['koi_disposition_encoded'], axis=1)

In [73]:
#split the data into training and testing sets
#NO suffix is for no outliers
X_train_NO, X_test_NO, y_train_NO, y_test_NO = train_test_split(X_not_outliers, y_no_outliers, test_size=0.2, random_state=42) #use a test size of 20%

Scale the data without outliers

In [74]:
#fit the scalers to the train and test sets separately and transform both

#standard scaling
X_train_stndrd_NO = scaler.fit_transform(X_train_NO)
X_test_stndrd_NO = scaler.transform(X_test_NO)

#minmax scaling
X_train_minmax_NO = minmax.fit_transform(X_train_NO)
X_test_minmax_NO = minmax.transform(X_test_NO)

In [75]:
#training our tree and ensemble models on the scaled data with outliers removed
#train Decision tree classifier
dtc.fit(X_train_stndrd_NO, y_train_NO)

#train Random forest classifier
rfc.fit(X_train_stndrd_NO, y_train_NO)

#train XGBoost classifier
xgbc.fit(X_train_minmax_NO, y_train_NO)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [76]:
#Training the remaining models on their preferred scaling method with outliers removed
#train Logistic regression classifier
logreg.fit(X_train_stndrd_NO, y_train_NO)

#train Support vector machine classifier
svc.fit(X_train_stndrd_NO, y_train_NO)

#train K-nearest neighbors classifier
knn.fit(X_train_stndrd_NO, y_train_NO)

KNeighborsClassifier()

Performance of models with outliers removed, and scaled data

In [77]:
#Decision tree classifier score
dtc_score = dtc.score(X_test_stndrd_NO, y_test_NO)

#Random forest classifier score
rfc_score = rfc.score(X_test_stndrd_NO, y_test_NO)

#Logistic regression classifier score
logreg_score = logreg.score(X_test_stndrd_NO, y_test_NO)

#Support vector machine classifier score
svc_score = svc.score(X_test_stndrd_NO, y_test_NO)

#K-nearest neighbors classifier score
knn_score = knn.score(X_test_minmax_NO, y_test_NO)

#XGBoost classifier score
xgbc_score = xgbc.score(X_test_stndrd_NO, y_test_NO)

In [78]:
#Print the scores

# Create a dictionary with the model names and their scores
model_scores_scaled_NO = {
    "Decision Tree": dtc_score,
    "Random Forest": rfc_score,
    "Logistic Regression": logreg_score,
    "SVM": svc_score,
    "KNN": knn_score,
    "XGBoost": xgbc_score
}

# Convert the dictionary to a DataFrame
scores_scaled_NO = pd.DataFrame(list(model_scores_scaled_NO.items()), columns=['Model-Scaling-No Outliers', 'Accuracy Score'])

# Sort the DataFrame by 'Accuracy Score' in descending order
scores_scaled_NO = scores_scaled_NO.sort_values(by='Accuracy Score', ascending=False)

# Display the DataFrame for both sets of scores
print(scores_df)
print(scores_scaled)
print(scores_scaled_NO)

                 Model  Accuracy Score
5              XGBoost        0.893452
1        Random Forest        0.891787
0        Decision Tree        0.849057
4                  KNN        0.623751
2  Logistic Regression        0.529967
3                  SVM        0.490566
         Model-Scaling  Accuracy Score
5              XGBoost        0.893452
1        Random Forest        0.892342
2  Logistic Regression        0.876249
3                  SVM        0.869589
0        Decision Tree        0.849057
4                  KNN        0.809101
  Model-Scaling-No Outliers  Accuracy Score
1             Random Forest        0.888140
2       Logistic Regression        0.882075
3                       SVM        0.879380
0             Decision Tree        0.828841
5                   XGBoost        0.671833
4                       KNN        0.598383


We saw drastic performance loss in some of our models, and even some performance loss in our top performaning models. This indicates that the so-called outliers, can not simply be filtered out using the methods that were employed. We will continue with the data as is, and look at other possibilities for preprocessing that may improve performance. 

### Logarithmic Scaling

As we saw in our data exploration, as well as when viewing the descriptive statistics of our cleaned dataset, there was a very wide range of values for many of the variables. Despite the scaling, these extreme values can still negatively impact our model performance. Another method for dealing with extreme values is log scaling. 

Log scaling also makes sense in the context of our data as it is measurements on an astronomical scale, where magnitudes can very logarithmically.

As there are no built in log-scalers, we can create our own. 

In [93]:
#Import relevant libraries
from sklearn.base import BaseEstimator, TransformerMixin

#Create a class to scale the data using the log function from numpy
class LogScaler(BaseEstimator, TransformerMixin):
    def __init__(self, epsilon=1e-5):
        self.epsilon = epsilon  # Small constant to handle zero values

    def fit(self, X, y=None):
        return self

    #ned to take the absolute value of the data to avoid errors
    def transform(self, X):
        return np.log1p(np.abs(X + self.epsilon))

In [94]:
#apply the log scaler to the data
log_scaler = LogScaler(epsilon=1e-5)

X_train_log = log_scaler.fit_transform(X_train)
X_test_log = log_scaler.transform(X_test)

We can train our models without our Standard or MinMax Scalers to compare performance:

In [96]:
#train Decision tree classifier
dtc.fit(X_train_log, y_train)

#train Random forest classifier
rfc.fit(X_train_log, y_train)

#train Logistic regression classifier
logreg.fit(X_train_log, y_train)

#train Support vector machine classifier
svc.fit(X_train_log, y_train)

#train K-nearest neighbors classifier
knn.fit(X_train_log, y_train)

#train XGBoost classifier
xgbc.fit(X_train_log, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [98]:
#Decision tree classifier score
dtc_score = dtc.score(X_test_log, y_test)

#Random forest classifier score
rfc_score = rfc.score(X_test_log, y_test)

#Logistic regression classifier score
logreg_score = logreg.score(X_test_log, y_test)

#Support vector machine classifier score
svc_score = svc.score(X_test_log, y_test)

#K-nearest neighbors classifier score
knn_score = knn.score(X_test_log, y_test)

#XGBoost classifier score
xgbc_score = xgbc.score(X_test_log, y_test)

Compare with our scaled data using normal transformers:

In [101]:
#Print the scores

# Create a dictionary with the model names and their scores
model_scores_log = {
    "Decision Tree": dtc_score,
    "Random Forest": rfc_score,
    "Logistic Regression": logreg_score,
    "SVM": svc_score,
    "KNN": knn_score,
    "XGBoost": xgbc_score
}

# Convert the dictionary to a DataFrame
scores_log = pd.DataFrame(list(model_scores_log.items()), columns=['Model-Log-Scaling', 'Accuracy Score'])

# Sort the DataFrame by 'Accuracy Score' in descending order
scores_log = scores_log.sort_values(by='Accuracy Score', ascending=False)

# Display the DataFrame for both sets of scores
print(scores_scaled)
print(scores_log)

print(classification_report(y_test, dtc.predict(X_test_log)))

         Model-Scaling  Accuracy Score
5              XGBoost        0.893452
1        Random Forest        0.892342
2  Logistic Regression        0.876249
3                  SVM        0.869589
0        Decision Tree        0.849057
4                  KNN        0.809101
     Model-Log-Scaling  Accuracy Score
5              XGBoost        0.896781
1        Random Forest        0.896226
2  Logistic Regression        0.894562
3                  SVM        0.865705
0        Decision Tree        0.846282
4                  KNN        0.798557
              precision    recall  f1-score   support

           0       0.72      0.70      0.71       452
           1       0.72      0.75      0.74       466
           2       0.98      0.97      0.97       884

    accuracy                           0.85      1802
   macro avg       0.81      0.81      0.81      1802
weighted avg       0.85      0.85      0.85      1802

